In [3]:
import pandas as pd 
import numpy as np 
from nltk.corpus import stopwords 
from sklearn.model_selection import train_test_split 
import tensorflow  as tf
import os 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding , LSTM , Dense , TextVectorization , Bidirectional 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json 



In [6]:
path = 'F:/deployed_book_review/deployed-bookReviews-inTheweb-withTensorflow'
true_data = pd.read_csv(os.path.join(path , 'data/True.csv'))
fake_data = pd.read_csv(os.path.join(path , 'data/Fake.csv'))


In [7]:
true_data['target'] = 1 
fake_data['target'] = 0
data = pd.concat([true_data , fake_data]) 
data.head()

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [8]:
X_train , X_val , y_train , y_val = train_test_split(data['text'] , data['target'] , test_size = .1  , shuffle = True ,random_state =123 ) 

In [9]:
MAX_FEATURES = 50000

In [10]:
tock = Tokenizer(num_words = MAX_FEATURES , oov_token = 'UNK')
tock.fit_on_texts(X_train)

In [11]:
X_train = tock.texts_to_sequences(X_train)
X_train = pad_sequences(X_train , maxlen=300)

In [12]:
X_val = tock.texts_to_sequences(X_val)
X_val = pad_sequences(X_val , maxlen=300)

In [41]:
input = tf.keras.Input(shape = (300 ,))
X = Embedding(50000 , 32 ) (input)
X = tf.keras.layers.Conv1D(32,3  , activation='relu')(X)
X = tf.keras.layers.GlobalAveragePooling1D()(X)
X = Dense(256  , activation='relu') (X)
X = Dense(64  , activation='relu')(X)
X = Dense(1 , activation ='sigmoid')(X)
model = tf.keras.Model(input , X)


In [43]:
model.compile(optimizer = 'adam' , loss ='binary_crossentropy' , metrics =['accuracy'])
model.fit(X_train ,y_train  ,validation_data=(X_val , y_val), epochs = 1  )

938/938 [==============================] - 45s 47ms/step - loss: 0.0289 - accuracy: 0.9915 - val_loss: 0.0249 - val_accuracy: 0.9938


In [36]:
with open('word_index.json' , 'w') as file : 
    json.dump(tock.word_index , file) 

In [45]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('lite_model_v1.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmplg83eq4n\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmplg83eq4n\assets
